### imports

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import time
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pickle

def create_submission(test_data, property_predicted_score):
    !mv submission.csv.zip prev.submission.zip
    test_data['raiting'] = property_predicted_score
    submition_data = test_data[['srch_id','prop_id','raiting']]
    submition_data = submition_data.sort_values(by=['srch_id', 'raiting'], ascending=[True,  False])
    submition_data = submition_data.drop(columns="raiting")
    submition_data.to_csv('submission.csv', index=False)
    !zip submission.csv.zip  submission.csv
    !rm submission.csv

### load training set 

In [13]:
#if loaded train file exists
try:
    with open('data/train-data.pickle', 'rb') as handle:
        train = pickle.load(handle)
#if not load and save
except:
    !unzip data/training_set_VU_DM.csv.zip
    train = pd.read_csv('training_set_VU_DM.csv')
    !rm training_set_VU_DM.csv
    with open('data/train-data.pickle', 'wb') as handle:
        pickle.dump(train, handle)

In [3]:
#if loaded test file exists
try:
    with open('data/test-data.pickle', 'rb') as handle:
        test = pickle.load(handle)
#if not load and save
except:
    !unzip data/test_set_VU_DM.csv.zip
    test = pd.read_csv('test_set_VU_DM.csv')
    !rm test_set_VU_DM.csv
    with open('data/test-data.pickle', 'wb') as handle:
        pickle.dump(test, handle)

In [4]:
def target_function(data_frame):
    return data_frame['click_bool'] + data_frame['booking_bool']*4 #if booked then clicked booked = 5

def features_engeneering(data_frame):
    data_frame.loc[data_frame['price_usd']==0,'price_usd'] = 100000
    data_frame['price_per_person'] = data_frame['price_usd']/(data_frame['srch_adults_count'] + data_frame['srch_children_count'])
    data_frame.loc[data_frame['prop_review_score'].isnull(),'prop_review_score'] = data_frame['prop_review_score'].median()
    return data_frame




### Total columns

In [5]:
len(train)

4958347

### Total null columns

In [14]:
print(train.isnull().sum())
print(train['prop_review_score'].median())
train.loc[train['prop_review_score'].isnull(),'prop_review_score'] = train['prop_review_score'].median()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating        4706481
visitor_hist_adr_usd           4705359
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7364
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1090348
prop_log_historical_price            0
position                             0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
srch_query_affinity_score      4640941
orig_destination_distance

In [7]:
df = train.dropna(axis=1)
del train
a = set(df.columns)
b = set(test.columns)
print(a-b)

{'position', 'booking_bool', 'click_bool'}


### split data to train and test

In [15]:
#print(df.columns)
df = features_engeneering(df)
test = features_engeneering(test)

features_to_choose = ['prop_review_score', 'prop_starrating', 'prop_brand_bool', 'prop_location_score1', 
                      'prop_log_historical_price', 'price_per_person',
                      'promotion_flag', 'srch_length_of_stay', 'srch_booking_window',
                      'srch_room_count', 'srch_saturday_night_bool']

X = df[features_to_choose]
Y = target_function(df)

X_to_test = test[features_to_choose]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
#kfold = KFold(n_splits=5)

NameError: name 'KFold' is not defined

In [9]:
from sklearn import neighbors, linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor
# Fit regression model
#n_neighbors = 10

#model = linear_model.LinearRegression()
#model = neighbors.KNeighborsRegressor(n_neighbors)
#model= DecisionTreeRegressor(random_state=0)
#model = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=5, random_state=0)
model = GradientBoostingRegressor(random_state=0)
###########################################
start = time.time()
model.fit(X_train, Y_train)
end = time.time()
print("The time of train fit:", end-start)

Y_pred = model.predict(X_test)
mae = mean_absolute_error(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred, squared=False)
print(f'mae: {mae} mse: {mse}')


The time of train fit: 411.9554362297058
mae: 0.2970567071906269 mse: 0.8276808065234093


In [10]:
start = time.time()
model.fit(X, Y)
end = time.time()
print("The time of final fit:", end-start)

raitings = model.predict(X_to_test)
create_submission(test, raitings)


The time of final fit: 518.7838761806488
mv: cannot stat 'submission.csv.zip': No such file or directory
  adding: submission.csv (deflated 67%)


In [11]:
create_submission(test, raitings)

  adding: submission.csv (deflated 67%)
